In [1]:
import re
import json
import time
import requests
import pymongo
import os.path
import warnings
warnings.filterwarnings('ignore')

client = pymongo.MongoClient(host='127.0.0.1',port=27017)
db = client['XY']
music_db = db['qq_music']



### Data Scraping

In [ ]:
URL = 'https://c.y.qq.com/base/fcgi-bin/fcg_global_comment_h5.fcg?'

HEADERS = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'
}

PARAMS = {
    'g_tk': '5381',
    'jsonpCallback': 'jsoncallback4823183319594757',
    'loginUin': '0',
    'hostUin': '0',
    'format': 'jsonp',
    'inCharset': 'utf8',
    'outCharset': 'GB2312',
    'notice': '0',
    'platform': 'yqq',
    'needNewCode': '0',
    'cid': '205360772',
    'reqtype': '2',
    'biztype': '1',
    'topid': '351042132',
    'cmd': '6',
    'needmusiccrit': '0',
    'pagenum': '0',
    'pagesize': '50',
    'lasthotcommentid': '',
    'callback': 'jsoncallback4823183319594757',
    'domain': 'qq.com',
    'ct': '24',
    'cv': '101010',
}

LAST_COMMENT_ID = ''


def get_html(url, headers, params=None, tries=3):
    try:
        response = requests.get(url=url, headers=headers, params=params)
        response.raise_for_status()
        response.encoding = 'utf-8'
    except requests.HTTPError:
        print("connect failed")
        if tries > 0:
            print("reconnect...")
            last_url = url
            get_html(last_url, headers, tries-1)
        else:
            print("3 times failure")
            return None
    return response

def paras_html(html):
    data = {}
    content = json.loads(html[29:-3])
    for item in content['comment']['commentlist']:
        data["nike"] = item.get("nick")
        data["comment"] = re.sub(r"\\n", " ", item.get("rootcommentcontent"))
        data["comment"] = (re.sub(r"\n", " ", data["comment"]))[0:255]
        data["praisenum"] = item.get("praisenum")
        data["comment_id"] = item.get("commentid")
        data["time"] = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(int(item.get("time"))))
        yield data



def get_top_comments(topid, m_name, tag):
    err_links = []
    lost_links = []
    comm_items = []
    count = 0
    PARAMS['topid'] = topid
    global LAST_COMMENT_ID
    for page in range(3):
        if page % 5 == 0:
            time.sleep(2)
        PARAMS['lastcommentid'] = LAST_COMMENT_ID
        PARAMS['pagenum'] = page
        if err_links:
            if count >= 3:
                lost_links.append(err_links.pop())
                continue
            count += 1
            response = get_html(err_links.pop(), HEADERS)
        else:
            count = 0
            response = get_html(URL, HEADERS, PARAMS)
        if response:
            try:
                for item in paras_html(response.text):
                    item['_id'] = item.pop('comment_id','')
                    item['m_name'] = m_name
                    item['tag'] = tag
                    item['topid'] = topid
                    music_db.update({'_id':item['_id']},item,upsert=True)
                    comm_items.append(item)
                    LAST_COMMENT_ID = item['_id']
            except TypeError:
                print(response.url)
                err_links.append(response.url)
    return comm_items 

def parse_mid(dissid):
    url = 'https://c.y.qq.com/qzone/fcg-bin/fcg_ucc_getcdinfo_byids_cp.fcg?'
    data = {
        'type':'1',
        'json':'1',
        'utf8':'1',
        'onlysong':'0',
        'new_format':'1',
        'disstid':dissid,
        'g_tk_new_20200303':'398486200',
        'g_tk':'398486200',
        'loginUin':'1781669751',
        'hostUin':'0',
        'format':'json',
        'inCharset':'utf8',
        'outCharset':'utf-8',
        'notice':'0',
        'platform':'yqq.json',
        'needNewCode':'0'
    }
    headers = {
        'accept': 'application/json,text/javascript,*/*;q=0.01',
        'accept-encoding': 'gzip,deflate,br',
        'accept-language': "zh-CN,zh;q=0.9",
        'cookie':'pgv_pvi=9467031552; RK=4ipsomknbT; ptcz=b5ecd4bbe3ce019acaaaa93bbdfa7c900a27314c2719747d2398955e31161e30; iip=0; o_cookie=1781669751; tvfe_boss_uuid=8cd31e438dfd07de; eas_sid=J1z6d2P476E0L4m7G1C0q0V342; fqm_pvqid=d98ac904-1d0d-42cb-ba88-b8c3dfd3ec65; _ga=GA1.2.24845414.1629536752; pgv_pvid=4719536160; pac_uid=1_1781669751; fqm_sessionid=df79dd6c-7efa-4fe8-bb71-40ae7641d799; pgv_info=ssid=s6172605620; ts_uid=7410441198; ts_last=y.qq.com/n/ryqq/playlist/8427125725; _qpsvr_localtk=0.8085239159623139; login_type=1; qqmusic_key=Q_H_L_51vgtWby8EKpld0DgGl5DHUtaSxoab3GtqR-8c9c5zApwP0FEEmmfyw; qm_keyst=Q_H_L_51vgtWby8EKpld0DgGl5DHUtaSxoab3GtqR-8c9c5zApwP0FEEmmfyw; psrf_qqopenid=72B9477B951FBE50F5D87B9A6D17DE9B; psrf_qqrefresh_token=949BA7E3BDEF485E0F2F88FF74DE210B; euin=oKSFoKCsNKSkov**; qm_keyst=Q_H_L_51vgtWby8EKpld0DgGl5DHUtaSxoab3GtqR-8c9c5zApwP0FEEmmfyw; psrf_musickey_createtime=1650182538; psrf_access_token_expiresAt=1657958538; wxrefresh_token=; uin=1781669751; wxunionid=; psrf_qqunionid=93ABBEEF8687B8136734BC3FD9B4C4AE; wxopenid=; tmeLoginType=2; psrf_qqaccess_token=EB938D468FB8E28D57B057A375AB5629',
        'origin':'https://y.qq.com',
        'referer': 'https://y.qq.com/',
        'sec-fetch-dest':'empty',
        'sec-fetch-mode':'cors',
        'sec-fetch-site':'same-site',
        'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'
    }
    try:
        response = requests.get(url=url , params=data , headers = headers)
        mid_list = []
        name_list = []
        mid_json = json.loads(response.text).get('cdlist')[0].get('songlist')
        for media_mid in mid_json:
            mid = media_mid.get('id')
            mid_list.append(mid)
            name = media_mid.get('name')
            name_list.append(name)
        return zip(mid_list,name_list)
    except requests.RequestException:
        print('error')





In [ ]:
play_dict = {}
pos_str = '''https://y.qq.com/n/ryqq/playlist/8427125725 (30)
https://y.qq.com/n/ryqq/playlist/8427713731 (21)
https://y.qq.com/n/ryqq/playlist/8433728400 (23)
https://y.qq.com/n/ryqq/playlist/7730682978 (21)
https://y.qq.com/n/ryqq/playlist/8433173166 (23)
https://y.qq.com/n/ryqq/playlist/8436440116 (31)
https://y.qq.com/n/ryqq/playlist/8433161117 (37)
https://y.qq.com/n/ryqq/playlist/8409780958 (66)'''
neg_str = '''https://y.qq.com/n/ryqq/playlist/8438392141 (36)
https://y.qq.com/n/ryqq/playlist/7779802947 (44)
https://y.qq.com/n/ryqq/playlist/8257217996 (314)'''
play_dict['pos'] = list(map(lambda x: x.split('/')[-1].split(' (')[0], pos_str.split('\n')))
play_dict['neg'] = list(map(lambda x: x.split('/')[-1].split(' (')[0], neg_str.split('\n')))
play_dict

{'neg': ['8438392141', '7779802947', '8257217996'],
 'pos': ['8427125725',
  '8427713731',
  '8433728400',
  '7730682978',
  '8433173166',
  '8436440116',
  '8433161117',
  '8409780958']}

In [ ]:
play_dict = {'pos':['8324867026','7466397730','7691838312','7071295880']}

In [ ]:
mid_tag_dict = {}
for tag in play_dict.keys():
    mid_sets = []
    for dissid in play_dict[tag]:
        mid_sets += list(parse_mid(dissid))
    mid_sets = list(set(mid_sets))
    mid_tag_dict[tag] = mid_sets

In [ ]:
len(mid_tag_dict['pos'])

283

In [ ]:
mid_tag_dict['pos'][10]

(407046, '日不落')

In [ ]:
#main get the top 100 comments of every song in every playlist. 
def main():
    
    for tag in list(mid_tag_dict.keys()):
        for i,tp in enumerate(mid_tag_dict[tag]):
            comm_items = get_top_comments(str(tp[0]),tp[1],tag)
            if i%5==0:
                time.sleep(2)
            print('success: %s -- %s'%(tp,len(comm_items)))
            
            
main()

https://c.y.qq.com/base/fcgi-bin/fcg_global_comment_h5.fcg?g_tk=5381&jsonpCallback=jsoncallback4823183319594757&loginUin=0&hostUin=0&format=jsonp&inCharset=utf8&outCharset=GB2312&notice=0&platform=yqq&needNewCode=0&cid=205360772&reqtype=2&biztype=1&topid=410314&cmd=6&needmusiccrit=0&pagenum=0&pagesize=50&lasthotcommentid=&callback=jsoncallback4823183319594757&domain=qq.com&ct=24&cv=101010&lastcommentid=
https://c.y.qq.com/base/fcgi-bin/fcg_global_comment_h5.fcg?g_tk=5381&jsonpCallback=jsoncallback4823183319594757&loginUin=0&hostUin=0&format=jsonp&inCharset=utf8&outCharset=GB2312&notice=0&platform=yqq&needNewCode=0&cid=205360772&reqtype=2&biztype=1&topid=410314&cmd=6&needmusiccrit=0&pagenum=0&pagesize=50&lasthotcommentid=&callback=jsoncallback4823183319594757&domain=qq.com&ct=24&cv=101010&lastcommentid=
https://c.y.qq.com/base/fcgi-bin/fcg_global_comment_h5.fcg?g_tk=5381&jsonpCallback=jsoncallback4823183319594757&loginUin=0&hostUin=0&format=jsonp&inCharset=utf8&outCharset=GB2312&notice=

In [12]:
#load data by mongodb
# comms = []
# for item  in music_db.find():
#     comms.append(item)
    
# len(comms)
#load data by json

with open('datasets.json','r', encoding='utf-8') as f:
    comms = json.loads(f.read())

    
len(comms)

31722

In [ ]:
# save to json
# with open('datasets.json','w',encoding='utf-8') as w:
#    w.write(json.dumps(comms,ensure_ascii=False))
    